In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [11]:
df_monthly = pd.read_csv('D:/AI-supply-chain-nestle/data/processed/cleaned_monthly_nestle.csv')
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly = df_monthly.sort_values('Date')

In [10]:
try:
    lstm_model = load_model('models/lstm_maggi.h5', custom_objects={'mse': 'mse'})
    print("Loaded base LSTM.")
except FileNotFoundError:
    print("Base model missing—re-run Phase 3 Step 3 to save 'lstm_maggi.h5'.")
    # Quick fallback: Skip predict, use historical avg for sim (strategy focus)
    future_forecast = np.full(6, df_monthly[df_monthly['Product'] == 'Maggi (Snacks)']['Monthly_Sales_Total'].mean())
    print("Fallback: Used historical avg for forecasts.")
else:
    # Prep future features (2025-01 to 06)
    future_dates = pd.date_range(start=df_monthly['Date'].max() + pd.DateOffset(months=1), periods=6, freq='MS')
    future_df = pd.DataFrame({'Date': future_dates})
    future_df['Monsoon_Month'] = future_df['Date'].dt.month.isin([7,8]).astype(int)
    future_df['Festival_Month'] = future_df['Date'].dt.month.isin([10,11,12]).astype(int)
    future_df['Promo_Proportion'] = 0.3
    future_df['Lag1_Sales'] = df_monthly[df_monthly['Product'] == 'Maggi (Snacks)']['Monthly_Sales_Total'].tail(1).values[0]
    future_df['Lag3_Sales'] = df_monthly[df_monthly['Product'] == 'Maggi (Snacks)']['Monthly_Sales_Total'].tail(3).mean()

Base model missing—re-run Phase 3 Step 3 to save 'lstm_maggi.h5'.
Fallback: Used historical avg for forecasts.


In [ ]:
historical_maggi = df_monthly[df_monthly['Product'] == 'Maggi (Snacks)'].groupby('Date').agg({
        'Monthly_Sales_Total': 'sum',
        'Monsoon_Month': 'max',
        'Festival_Month': 'max',
        'Promo_Proportion': 'mean'
    }).reset_index()
    historical_maggi['Lag1_Sales'] = historical_maggi['Monthly_Sales_Total'].shift(1)
    historical_maggi['Lag3_Sales'] = historical_maggi['Monthly_Sales_Total'].shift(3)
    historical_maggi = historical_maggi.dropna()
    features = ['Monsoon_Month', 'Festival_Month', 'Promo_Proportion', 'Lag1_Sales', 'Lag3_Sales']
    X_hist = historical_maggi[features]
    y_hist = historical_maggi['Monthly_Sales_Total']
    scaler_X = MinMaxScaler().fit(X_hist)
    scaler_y = MinMaxScaler().fit(y_hist.values.reshape(-1, 1))

    future_X = scaler_X.transform(future_df[features])
    future_X_lstm = future_X.reshape((future_X.shape[0], 1, future_X.shape[1]))
    future_forecast_scaled = lstm_model.predict(future_X_lstm)
    future_forecast = scaler_y.inverse_transform(future_forecast_scaled).ravel()

IndentationError: unexpected indent (944508290.py, line 7)